In [1]:
import pandas as pd 
import pickle 


In [2]:
with open("madelon.pkl",'rb') as f:
    results=pickle.load(f)

In [3]:
# for res in results

In [4]:

pd.DataFrame.from_dict(results)

,0,1,2,3,4
0,{'f_classif': {'fs_time': 0.012427221983671188...,{'f_classif': {'fs_time': 0.011810168623924255...,{'f_classif': {'fs_time': 0.029073424637317657...,{'f_classif': {'fs_time': 0.015629496425390244...,{'f_classif': {'fs_time': 0.020147234201431274...
1,{'SelectFdr': {'fs_time': 0.001269120723009109...,"{'SelectFdr': {'fs_time': 0.0012517049908638, ...",{'SelectFdr': {'fs_time': 0.007635641843080521...,{'SelectFdr': {'fs_time': 0.001146566122770309...,{'SelectFdr': {'fs_time': 0.001602593809366226...


In [5]:
results.keys()

dict_keys([0, 1, 2, 3, 4])

In [6]:
from dis import dis


def clf_res(res,clf):
    # res= k_value_dict[k][clf]
    clf_df= pd.DataFrame.from_dict(res)
    clf_df.index = clf_df.index.set_names(['metric'])
    clf_df.reset_index(inplace=True)
    clf_df["Learning algorithm"]= clf
    
    return clf_df

def turn_resDict_to_df(results):
    all_df=[]
    for fold,fs_algo_lst in results.items():
        for fs_algo in fs_algo_lst:
            for fs_algo_name in fs_algo.keys():
                k_value_dict= fs_algo[fs_algo_name]
                k_df= pd.DataFrame()
                fs_time= k_value_dict["fs_time"]
                for k in list(k_value_dict.keys())[1:]:
                
                    res_df= pd.DataFrame()
                    res_df=map(lambda clf:(clf_res(k_value_dict[k][clf],clf)),list(k_value_dict[k].keys())[2:])
                    res_df= pd.concat(res_df)
                    # display(res_df)
                    chosen_features= k_value_dict[k]["chosen_features"]
                    feature_rank= k_value_dict[k]["feature_rank"]
                    res_df["chosen_features"]=[chosen_features]*res_df.shape[0]
                    res_df["Selected Features scores"]=[feature_rank]*res_df.shape[0]
                    res_df["fs_time"]=[fs_time]*res_df.shape[0]
                    res_df["Number of features selected (K)"]=[k]*res_df.shape[0]
                    res_df["Filtering Algorithm"]=[fs_algo_name]*res_df.shape[0]
                    res_df["Fold"]=[fold]*res_df.shape[0]
                    all_df.append(res_df.copy(deep=True))
    return pd.concat(all_df, ignore_index=True)
# turn_resDict_to_df(results)

In [7]:
with open("results/scikit_COIL20.pkl",'rb') as f:
    results=pickle.load(f)

In [10]:
results[results["Filtering Algorithm"]=="f_classif"]

,metric,proba_time,res,Learning algorithm,chosen_features,Selected Features scores,fs_time,Number of features selected (K),Filtering Algorithm,Fold,database
0,AUC,0.069324,0.440154,LogisticRegression,[1023],[9492.971303982802],0.050150,1,f_classif,0,scikit_COIL20
1,MCC,0.069324,0.117197,LogisticRegression,[1023],[9492.971303982802],0.050150,1,f_classif,0,scikit_COIL20
2,PR-AUC,0.069324,0.079441,LogisticRegression,[1023],[9492.971303982802],0.050150,1,f_classif,0,scikit_COIL20
3,accuracy,0.069324,0.097222,LogisticRegression,[1023],[9492.971303982802],0.050150,1,f_classif,0,scikit_COIL20
4,AUC,0.121026,0.216745,RandomForestClassifier,[1023],[9492.971303982802],0.050150,1,f_classif,0,scikit_COIL20
...,...,...,...,...,...,...,...,...,...,...,...
2155,accuracy,0.024852,0.656250,GaussianNB,"[1023, 0, 1022, 992, 31, 960, 32, 30, 1, 991, ...","[9573.765987026421, 7332.850692020083, 6411.75...",0.109516,100,f_classif,4,scikit_COIL20
2156,AUC,0.181221,0.524790,SVC,"[1023, 0, 1022, 992, 31, 960, 32, 30, 1, 991, ...","[9573.765987026421, 7332.850692020083, 6411.75...",0.109516,100,f_classif,4,scikit_COIL20
2157,MCC,0.181221,0.761921,SVC,"[1023, 0, 1022, 992, 31, 960, 32, 30, 1, 991, ...","[9573.765987026421, 7332.850692020083, 6411.75...",0.109516,100,f_classif,4,scikit_COIL20
2158,PR-AUC,0.181221,0.045759,SVC,"[1023, 0, 1022, 992, 31, 960, 32, 30, 1, 991, ...","[9573.765987026421, 7332.850692020083, 6411.75...",0.109516,100,f_classif,4,scikit_COIL20
